In [1]:
import math
import json
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import sys
sys.path.append("../")

from unique_hypercube_size import *

%load_ext autoreload
%autoreload 2


In [2]:
"""

    Set up the parameters
    
"""

num_trials = 20

phase_resolution = 0.3


m_max = 10
k_max = 10

ms     = range(1, m_max + 1)
ks     = range(1, k_max + 1)
scales = np.sqrt(2.)**np.arange(m_max)
# scales = 1. + np.random.sample(m_max)*0.5


print scales

[  1.           1.41421356   2.           2.82842712   4.           5.65685425
   8.          11.3137085   16.          22.627417  ]


In [ ]:
"""

    Run the experiments and 
    collect the data:
    
        - width 
        - det(R)  
        - volume of embedded unit cube
    
"""

emb_vol = np.zeros((m_max, k_max, num_trials))
width   = np.zeros((m_max, k_max, num_trials))
det_R   = np.zeros((m_max, k_max, num_trials))


for t in xrange(num_trials):
    A, results = doRandomModuleExperiment(ms, ks, scales, phase_resolution)
    
    for (m,k), r in results.iteritems():
        width[m-1,k-1, t] = r
    
    for m in ms:
        for k in ks:
            if 2*m >= k:
                Q, R   = np.linalg.qr(A[:m,:,:k].reshape((2*m,k)))
                
                det_R[m-1,k-1,t] = np.absolute(np.linalg.det(R))
                
                vol_k     = det_R[m-1,k-1,t]
                vol_fibre = phase_resolution**(2*m - k)
                
                emb_vol[m-1,k-1,t] = vol_k * vol_fibre

    

        




1 1
1 2
1 3
1 4
1 5
1 6
2 1
2 2
2 3
2 4
2 5
2 6
3 1
3 2
3 3
3 4
3 5
3 6
4 1
4 2
4 3
4 4
4 5
4 6
5 1
5 2
5 3
5 4
5 5
5 6
6 1
6 2
6 3
6 4
6 5
6 6
7 1
7 2
7 3
7 4
7 5
7 6
8 1
8 2
8 3
8 4
8 5
8 6
9 1
9 2
9 3
9 4
9 5
9 6
10 1
10 2
10 3
10 4
10 5
10 6
1 1
1 2
1 3
1 4
1 5
1 6
2 1
2 2
2 3
2 4
2 5
2 6
3 1
3 2
3 3
3 4
3 5
3 6
4 1
4 2
4 3
4 4
4 5
4 6
5 1
5 2
5 3
5 4
5 5
5 6
6 1
6 2
6 3
6 4
6 5
6 6
7 1
7 2
7 3
7 4
7 5
7 6
8 1
8 2
8 3
8 4
8 5
8 6
9 1
9 2
9 3
9 4
9 5
9 6
10 1
10 2
10 3
10 4
10 5
10 6
1 1
1 2
1 3
1 4
1 5
1 6
2 1
2 2
2 3
2 4
2 5
2 6
3 1
3 2
3 3
3 4
3 5
3 6
4 1
4 2
4 3
4 4
4 5
4 6
5 1
5 2
5 3
5 4
5 5
5 6
6 1
6 2
6 3
6 4
6 5
6 6
7 1
7 2
7 3
7 4
7 5
7 6
8 1
8 2
8 3
8 4
8 5
8 6
9 1
9 2
9 3
9 4
9 5
9 6
10 1
10 2
10 3
10 4
10 5
10 6
1 1
1 2
1 3
1 4
1 5
1 6
2 1
2 2
2 3
2 4
2 5
2 6
3 1
3 2
3 3
3 4
3 5
3 6
4 1
4 2
4 3
4 4
4 5
4 6
5 1
5 2
5 3
5 4
5 5
5 6
6 1
6 2
6 3
6 4
6 5
6 6
7 1
7 2
7 3
7 4
7 5
7 6
8 1
8 2
8 3
8 4
8 5
8 6
9 1
9 2
9 3
9 4
9 5
9 6
10 1
10 2
10 3
10 4
10 5
10 6
1 1
1 2
1 3
1 4


In [92]:
"""

    Compute the quantities derived
    from data
    
"""



mean_width   = np.zeros((m_max, k_max))
vol          = np.zeros((m_max, k_max, num_trials))
mean_vol     = np.zeros((m_max, k_max))
mean_emb_vol = np.zeros((m_max, k_max))
mean_det_R   = np.zeros((m_max, k_max))

for m in ms:
    for k in ks:
        m_ = m - 1
        k_ = k - 1
        vol[m_,k_]          = width[m_,k_]**k
        mean_width[m_,k_]   = np.mean(width[m_,k_])
        mean_vol[m_,k_]     = np.mean(vol[m_,k_])
        mean_emb_vol[m_,k_] = np.mean(emb_vol[m_,k_])
        mean_det_R[m_,k_]   = np.mean(det_R[m_,k_])

        

In [93]:
"""
    
    Compute the volumes for the 
    modules-per-axis strategy
    
"""

inferred_vol   = np.zeros((m_max,k_max))
inferred_width = np.zeros((m_max,k_max))

for m in ms:
    m_ = m - 1
    for k in ks:
        k_  = k - 1
        mx  = float(m)/float(k)
        m0 = 1
        m1 = 9
        
        assert mean_width[m0, 0]>0 and mean_width[m1, 0]>0, \
            "The 1-d case might be missing"
            
        r0 = mean_width[m0, 0]
        r1 = mean_width[m1, 0]

        offset = m0 + 1.
        slope  = (np.log10(r1) - np.log10(r0))/(float(m1)-float(m0))
        base   = np.log10(r0)
        
        rx = 10**(base + (mx - offset)*slope)
        vx = rx**k
        inferred_width[m_, k_] = rx
        inferred_vol[m_,k_] = vx




AssertionError: The 1-d case might be missing

In [94]:
"""
    
    Save the data
    
"""

import datetime
timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H:%M:%S')


exp_folder = "../results"
exp_tag    = "m{}_k{}_t{}_shuff_A__{}".format(m_max, k_max, num_trials, timestamp)

# filename = "{}/cap_data_{}.p".format(exp_folder, exp_tag)
filename = "{}/temp.p".format(exp_folder)

print filename

with open(filename,'wb') as f:
    pickle.dump({
        "width": width,
        "vol": vol,
        "inferred_vol": inferred_vol,
        "inferred_width": inferred_width,
        "emb_vol": emb_vol,
        "det_R": det_R,
        "mean_width": mean_width,
        "mean_vol": mean_vol,
        "mean_emb_vol": mean_emb_vol,
        "mean_det_R": mean_det_R,
        "ms": ms,
        "ks": ks,
        "phase_resolution": phase_resolution
    }, f)

../results/temp.p
